In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tn

In [ ]:
#data_standart = pd.read_csv('year_transaction_standart.csv')
#data_standart_year_1 = pd.read_csv('year_transaction_1.csv')
#data_standart_year_2_test = pd.read_csv('year_transaction_2.csv')
ideal_y_vector = pd.read_csv('terminals.csv',header=None)
ideal_y_vector.columns=['nomber_terminal','money']

In [ ]:
ideal_y_vector.head()

data_standart.shape[0]
data_standart_year_1.shape[0]
data_standart_year_2_test.shape[0]

In [ ]:
from tqdm import tqdm_notebook as tn
def rewrite_file(name_file):
    '''т.к. изначальная бд была представленна не очень корректно в плане времени и даты(одним столбцом), 
    что крайне неудобно было при сортировке, было принято решение, улучшить DataFrame'''
    f = open(name_file,'r')
    g = open('new_'+name_file,'w')
    for line in tn(f):
        if line[0] == ',':
            p = line[1:len(line)].split(',')
            g.write(p[0]+','+p[1]+','+'month'+','+'day'+','+p[3]+','+p[4]+','+p[5]+','+p[6]+','+p[7])
            g.write('\n')
        else:
            p = line[1:len(line)].split(',')
            y = p[3].split('.')
            g.write(p[1]+','+p[2]+','+y[1]+','+y[0]+','+p[4]+','+p[5]+','+p[6]+','+p[7]+','+p[8])
            g.write('\n')
    f.close()
    g.close()
    print('запись завершена')

1. Подготовка выборк – для этого необходимо рассчитать отток загружаемых в банкомат средств: все транзакции с меткой operation “-”

! мы не берем транзакции '/', т.к. они лишь перенос средств с одного абстрактного счета на другой, без изятия средств банкомата.

In [ ]:
#комп не выдерживает загрузки 3-х файлов сразу
array_data = ['year_transaction_standart.csv','year_transaction_1.csv','year_transaction_2.csv']
for i,x in tn(enumerate(array_data)):
    rewrite_file(x)#изменение DataFrame
    new_data_standart = pd.read_csv('new_'+x)
    #data = new_data_standart[new_data_standart['operation'] == '-'].groupby(['nomber_terminal','month'])['money'].sum()
    data = new_data_standart[new_data_standart['operation'] == '-'].groupby(['nomber_terminal'])['money'].sum()
    data.to_csv('lose_money_'+str(i)+'.csv')
print('Все приготовления и приведения БД в порядок завершены!')

2. Проверка всех выборк в сравнении с идеальной выборкой по банкоматам: Точность ≈ %

In [ ]:
#1 год
data_1 = pd.read_csv('lose_money_0.csv',header=None)
data_1.columns=['nomber_terminal','money']
#data_1.columns=['nomber_terminal','month','money']
data_1.head()

In [ ]:
#2 год
data_2 = pd.read_csv('lose_money_1.csv',header=None)
data_2.columns=['nomber_terminal','money']

In [ ]:
#3 год
data_3 = pd.read_csv('lose_money_2.csv',header=None)
data_3.columns=['nomber_terminal','money']

Нас не интересует, если денег из банкомата снимает меньше нормы, а вот если есть превышение, тогда это считается ошибкой.
Чем ниже процент, тем ниже общая ошибка

In [ ]:
def accuracy_error(data,data_ideal):
    data_new = []
    kol = 0
    for x in range(data.shape[0]-1):
        if data_ideal.loc[x][1] - data.loc[x][1] < 0 : kol+=1
        #data_new.append([x,data_ideal.loc[x][1] - data.loc[x][1]])
    print('Ошибка '+str(100/data_ideal.shape[0]*kol)+' %') 
    #return data_new

In [ ]:
# В данном случае процент ошибки = 0, это значит что идеальное наполнение банкоматов устанавливалось по этому году
accuracy_error(data_1,ideal_y_vector)

2.1 Проверка 2 и 3 года выборки в сравнении с идеальной(1) выборкой по банкоматам

In [ ]:
accuracy_error(data_1,data_2)

In [ ]:
accuracy_error(data_1,data_3)

In [ ]:
accuracy_error(data_2,data_3)

# Вывод
    Самую высокую ошибку 50.2% от стандартного года(data_1) имеет 2 год(data_2) поэтому стоит рассмотреть именно его в сравнении со стандартным, для понимания протекающих изменений

3. Проверка разницы транзакций стандартного года с последующими

In [ ]:
%%time
new_data_standart_1 = pd.read_csv('new_year_transaction_standart.csv')
new_data_standart_2 = pd.read_csv('new_year_transaction_1.csv')

In [ ]:
#Слишком долгие вычисления, но без нагрузки памяти и перегрузки оперативы
def check_transaction(data,data_1, Return_data = False):
    distortion = 0
    percent = 0
    for x in tn(range(data.shape[0])):
        if sum( data.loc[x] == data_1.loc[x] ) != 9:
            array_of_distortion.append([data.loc[x],data_1.loc[x]])
            distortion+=1
        #else:
            #print(s,':',data.loc[x])
    percent = 100/data.shape[0]*distortion
    print("Отклонение транзакций – {0}%; Измененных {1}".format(percent,distortion))
    if Return_data:
        return array_of_distortion,distortion,percent

In [ ]:
#check_transaction(new_data_standart_1,new_data_standart_2)

4. Исключения. Создание нового Data Frame – отклоняющиеся транзакции(которые изменены для проверяемых годов)

In [ ]:
#Данные действия в функции серьезно нагружают вычислительные мощности компьютера и забивают оперативную память
def check_transaction(data,data_1, Return_data = False):
    common_cols = data.columns.tolist()
    array_copy = pd.merge(data, data_1, on=common_cols, how='inner')
    array_of_distortion = data_1[~data_1['nomber_transaction'].isin(array_copy['nomber_transaction'])]
    array_of_distortion_1 = data[~data['nomber_transaction'].isin(array_copy['nomber_transaction'])]
    distortion = array_of_distortion.shape[0]
    percent = 100/data.shape[0]*distortion
    print("Отклонение транзакций – {0}%; Измененных {1}".format(percent,distortion))
    if Return_data:
        return array_copy,array_of_distortion,array_of_distortion_1,distortion,percent

In [ ]:
array_copy,array_of_distortion,array_of_distortion_1,distortion,percent = check_transaction(new_data_standart_1,new_data_standart_2, Return_data = True)
array_of_distortion.to_csv('array_of_distortion.csv')
array_of_distortion_1.to_csv('array_of_distortion_1.csv')

Отклонение транзакций – 10%; Измененных 950.673

Сравним таким же образом стандартный и 3 год, а также 2 и 3. На самом деле, этого можно не делать, ведь сравнение стандартного и 2 года дают практический исчерпывающий ответ, но для определения зависимостей, мы все-таки их рассмотрим

In [ ]:
%time
new_data_standart_2 = pd.read_csv('new_year_transaction_2.csv')

In [ ]:
check_transaction(new_data_standart_1,new_data_standart_2)

In [ ]:
%time
new_data_standart_1 = pd.read_csv('new_year_transaction_1.csv')

In [ ]:
check_transaction(new_data_standart_2,new_data_standart_1)

5. Изучение искажений
<ol>
    <li>Nomber_terminal – 2%</li>
    <li>Nomber_transaction – не влияет на транзакцию</li>
    <li>Day_time – 70% ~ 30%</li>
    <li>Type_transaction and operation – 20%</li>
    <li>Id_card_1 and id_card_2 – 3%</li>
    <li>Money – 5%</li>
</ol>

<div>Группировка всех транзакций по пользователям, выделение % искажения транзакций для каждого пользователя.</div>
<div>Для того, чтобы решить эту задачу, нам нужно узнать количество всех транзакций за 2 года для каждого пользователя,</div>
<div>узнать количество транзакций измененных на следующий год(неважно, какое искажение было задействованно)</div>
<div>После чего узнать % искаженных транзакций от общего количества у каждого пользователя.</div>

In [ ]:
#2 года
%%time
new_data_standart_1 = pd.read_csv('new_year_transaction_standart.csv')
new_data_standart_2 = pd.read_csv('new_year_transaction_1.csv')
#2 датафрейма с искажаемыми транзакциями, полученные из пункта 4 array_of_distortion,array_of_distortion_1
array_of_distortion = pd.read_csv('array_of_distortion.csv')
array_of_distortion_1 = pd.read_csv('array_of_distortion_1.csv')
#пустой DataFrame куда и будем складывать все вычисления data

<p>выбираем всех пользователей из 4-х DataFrame, считая сколько раз они встречаются, после чего объединяем их по индексу id_user</p>
<p>после объединения, делаем новыую индексацию, и задаеи название колонок</p>

In [ ]:
data = pd.DataFrame(data=pd.concat([new_data_standart_1['id_card_1'].value_counts(),new_data_standart_2['id_card_1'].value_counts(),array_of_distortion['id_card_1'].value_counts(),array_of_distortion_1['id_card_1'].value_counts()], axis=1).reset_index(), 
                    columns=['id_user','year_1_all_transaction','year_2_all_transaction','year_1_lose_transaction','year_2_lose_transaction'])
data.stack().apply(pd.to_numeric, errors='ignore').fillna(0).unstack() #заменяю пустые None ячейки на 0

создаем 3 новых столбца 
1. % искажений 1 года к 1 году
2. % искажений 2 года к 2 году
3. от общего процент искажений(2 новых столбеца 1 + 2 год)

In [ ]:
data['percent_year_1'] = data.apply(lambda row: 100/row.year_1_all_transaction*row.year_1_lose_transaction, axis=1)
data['percent_year_1'] = data.apply(lambda row: 100/row.year_1_all_transaction*row.year_1_lose_transaction, axis=1)
data['sum_year'] = data.apply(lambda row: row.year_1_all_transaction + row.year_2_all_transaction, axis=1)
data['sum_lose'] = data.apply(lambda row: row.year_1_lose_transaction + row.year_2_lose_transaction, axis=1)
data['percent_year_1'] = data.apply(lambda row: 100/row.sum_year*row.sum_lose, axis=1)
data.head()

<p>Не забываем также учитывать пользователей с одной единственной транзакцией, которых нет в следующем году(отказались от наших услуг, мошенники, которые используют счета для перевода и отмывания средств, пользователи использующие другие банковские системы и др.) смотрите 'дополнительно' в конце notebook</p>

Сложение средств из неизменяемых транзакций

Используя функцию check_transaction, мы получили DataFrame array_copy в котором храняться одинаковые пересекающиеся транзакции 2-х лет, поэтому сравним его с идеальным годом и посмотрим какой процент ошибок наполнения возникает

In [ ]:
array_copy.to_csv('array_copy.csv')
#data = array_copy[array_copy['operation'] == '-'].groupby(['nomber_terminal','month'])['money'].sum()
data = array_copy[array_copy['operation'] == '-'].groupby(['nomber_terminal'])['money'].sum()
i = 4
data.to_csv('lose_money_'+str(i)+'.csv')
print('Преобразование завершено!')
accuracy_error(data,data_1)

# Результат

Построение модели основываясь на первом годе(стандартном) ≈ 90%
Простое сложение отрицательного накопления средств для одного банкомата за неделю


In [ ]:
data.head() файл 'lose_money_4.csv' будет ответом насколько необходимо наполнять банкоматы в год
если же взять файл 'array_copy.csv', то можно рассчитать сколько необходимо напонять банкоматы в течении дня, недели, месяца

В год data = array_copy[array_copy['operation'] == '-'].groupby(['nomber_terminal'])['money'].sum()

В день data = array_copy[array_copy['operation'] == '-'].groupby(['nomber_terminal','day'])['money'].sum()

Вычислить для недели сложней, т.к. мы не знаем какой год и не знаем как правильно будет сгруппировать по числам

В месяц data = array_copy[array_copy['operation'] == '-'].groupby(['nomber_terminal','month'])['money'].sum()

# Дополнительно

В дальнейшем, для более точного ответа, можно учитывать пользователей, которые постоянно изменяют транзакции. Сделать их классификацию и разбить по классам. Но для этого нужно выделить параметры классов, проще говоря можно использовать метод ближайших соседей в нескольких измерениях...
Ниже приведен пример DataFrame, который можно использовать для данного метода
!сделал маленький, похожий на настоящий из-за того, что компьютер не потянет 1гб таких вычислений.

# Построение модели основанной на классе пользователя.
<p>Если пользователь относится к классу с малым количеством транзакций(1-2 за несколько лет), то исключаем его из вычисления.</p>
<p>Класс 1: 1-2</p>
<p>Класс 2: 2-20</p>
<p>Класс 3: 20-90</p>
<p>Класс 4: 90+</p>

In [ ]:
#id_user - содержится id пользователей
#year_т - соержится список всех транзакций пользователя в этот год
import random as rd
def create_data(N):
    f = open('KNN_user_small.csv','w')
    f.write('id_user,year_1,year_2,year_3,class_KNN')
    f.write('\n')
    for x in tn(range(N)):
        a = rd.randint(0,100)
        b = rd.randint(0,100)
        c = rd.randint(0,100)
        sum_all = sum([a,b,c])
        if (sum_all>=1)and(sum_all<=2):
            class_KNN = '1'
        elif (sum_all>=2)and(sum_all<=20):
            class_KNN = '2'
        elif (sum_all>=20)and(sum_all<=90):
            class_KNN = '3'
        elif (sum_all>=90):
            class_KNN = '4'
        f.write(str(x)+','+str(a)+','+str(b)+','+str(c)+','+class_KNN)
        f.write('\n')
    f.close()

In [ ]:
create_data(10000)
data = pd.read_csv('KNN_user_small.csv')
data.head()

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

y = data['class_KNN']
X_train, X_holdout, y_train, y_holdout = train_test_split(data.values, y, test_size=0.3,random_state=17)

tree = DecisionTreeClassifier(max_depth=5, random_state=17)
knn = KNeighborsClassifier(n_neighbors=10)

tree.fit(X_train, y_train)
knn.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
tree_pred = tree.predict(X_holdout)
accuracy_score(y_holdout, tree_pred)

In [ ]:
knn_pred = knn.predict(X_holdout)
accuracy_score(y_holdout, knn_pred)

В конечном итоге на игрушечной выборке, наилучшим методом классификации стал метод решающих деревьев.